In [ ]:
import MySQLdb

### A. MySQL or mariaDB

#### 1. mariadb database생성
```sql
create database mysuppliers;

show database
;

use mysupplirs;

create table if not exists suppliers(
     supplier_name    varchar(20)
   , invoice_number   varchar(20)
   , part_number      varchar(20)
   , cost             float
   , purchase_date    date
);

describe suppliers; 
desc suppliers;
```
#### 2. DB접속을 위한 패키지 설치
* MySQL or Mariadb와 연동을 하기 위해서 `mysqlclient`패키지를 설치해야 한다.
>* pip install mysqlclient
>* conda install -y mysqlclient

##### 설치순서
1. pip install mysqlclient
2. python -m pip install --upgrade pip
3. pip install mysqlclient --upgrade
4. pip list or pip show mysqlclient

In [ ]:
!pip install mysqlclient
!pip show mysqlclient

In [ ]:
import csv
import MySQLdb  # mysql, mariadb 연동모듈
from datetime import datetime, date

print(dir(MySQLdb))

#### 1. MySQL 연동
##### 1. 레코드조회하기

In [ ]:
# mysql(3306) or mariadb(3307) 연결
conn = MySQLdb.connect(host="localhost", port=3307, db='mysuppliers', user='root', password='12345')
print(dir(conn))
print()
# conn.cursor?
cursor = conn.cursor()
print(dir(cursor))
print()

# sql
sql = 'select * from suppliers'
cursor.execute(sql)
suppliers = cursor.fetchall()

for supplier in suppliers:
    output = []
    for i in range(len(supplier)):
        output.append(str(supplier[i]))
    print(output)

cursor.close()
conn.close()    

In [ ]:
# 1. csv파일을 읽어서 suppliers에 insert하기

# 1) db접속
conn = MySQLdb.connect(host="localhost", port=3307, db='mysuppliers', user='root', password='12345')
cursor = conn.cursor()

# 2) csv파일 읽기
f_name = './data/suppliers.csv'
f_reader = csv.reader(open(f_name, 'r'), delimiter=',')
print(type(f_reader), f_reader)

# 3) header 처리
header = next(f_reader)
print(type(header), header)

for row in f_reader:
    # print(type(row), row)
    data = []
    for i in range(len(header)):
        if i<4: # Purchase Date(idx=4)
            data.append(str(row[i].replace(',','')).strip())
        else:
            # strptime(문자열, 형식)
            # strftime() : 날짜를 문자열로 변환함수
            p_date = datetime.date(datetime.strptime(str(row[i]), '%Y-%m-%d'))
            p_date = p_date.strftime('%Y-%m-%d')
            data.append(p_date)
    print(data)
    cursor.execute('insert into suppliers values(%s,%s,%s,%s,%s)', data)                           

cursor.close()
conn.commit()
conn.close()

In [ ]:
# 2. suppliers를 읽어서 출력해보기
conn = MySQLdb.connect(host="localhost", port=3307, db='mysuppliers', user='root', password='12345')
cursor = conn.cursor()
sql = 'select * from suppliers'

cursor.execute(sql)
suppliers = cursor.fetchall()

for supplier in suppliers:
    output = []
    for i in range(len(supplier)):
        output.append(str(supplier[i]))
    print(output)

cursor.close()
conn.close()   

In [ ]:
# 3. supplier name이 A인 자료만 수정
# cursor.exeecute(update-sql, (150, '오늘날짜', A))
conn = MySQLdb.connect(host="localhost", port=3307, db='mysuppliers', user='root', password='12345')
cursor = conn.cursor()
sql = 'update suppliers set cost=%s, purchase_date=%s where supplier_name = %s'
cursor.execute(sql, (150, '2024-03-22', 'A'))
conn.commit()
               
sql = 'select * from suppliers'

cursor.execute(sql)
suppliers = cursor.fetchall()

for supplier in suppliers:
    output = []
    for i in range(len(supplier)):
        output.append(str(supplier[i]))
    print(output)

cursor.close()
conn.close()   

In [ ]:
# 4. supplier name이 C인 자료만 삭제
conn = MySQLdb.connect(host="localhost", port=3307, db='mysuppliers', user='root', password='12345')
cursor = conn.cursor()
sql = 'delete from suppliers where supplier_name = %s'
cursor.execute(sql, ('C',))
conn.commit()

sql = 'select * from suppliers'
cursor.execute(sql)
suppliers = cursor.fetchall()

for supplier in suppliers:
    output = []
    for i in range(len(supplier)):
        output.append(str(supplier[i]))
    print(output)

cursor.close()
conn.close()  

In [ ]:
# 5. csv파일 출력하기
conn = MySQLdb.connect(host="localhost", port=3307, db='mysuppliers', user='root', password='12345')
cursor = conn.cursor()
cursor.execute('select * from suppliers where cost > 150')
rows = cursor.fetchall()

o_file = './data/suppliers_output.csv'
with open(o_file, 'w', newline='') as f:
    fw = csv.writer(f)  # empty file

    # 제목행
    header = ['Supplier Name', ' Invoice Number', ' Part Number', ' Cost', ' Purchase Date']
    fw.writerow(header)

    for row in rows:
        fw.writerow(row)

cursor.close()
conn.close()

### B. Oracle

#### 1. oracle package
>* python -m pip install cx_Oracle --upgrade
>* cx_Oracle패키지가 python-oracledb로 변경
>* https://oracle.github.io/python-cx_Oracle

#### python package repository
* https://pypi.org

#### 2. scott에 table생성

```sql
create table product(
    product_id number,
    product_name varchar2(50),
    price number default 0,
    decription clob,
    picture_url varchar2(500),
    primary key(product_id)
);

insert into product select 1, '레몬', 1500, '레몬에 포함된 구연산은 피로회복에... ', 'lemon.jpg' from dual;
insert into product select 2, '오렌지', 2000, '오렌지에 포함된 구연산은 피로회복에. ', 'orange.jpg' from dual;
insert into product select 3, '키위', 2500, '키위에 포함된 구연산은 피로회복에... ', 'kiwi.jpg' from dual;
insert into product select 4, '포도', 3000, '포도에 포함된 구연산은 피로회복에... ', 'grape.jpg' from dual;

commit;
```

In [ ]:
!python -m pip install cx_Oracle --upgrade

In [16]:
import cx_Oracle
print(dir(cx_Oracle))

['ATTR_PURITY_DEFAULT', 'ATTR_PURITY_NEW', 'ATTR_PURITY_SELF', 'ApiType', 'BFILE', 'BINARY', 'BLOB', 'BOOLEAN', 'Binary', 'CLOB', 'CURSOR', 'Connection', 'Cursor', 'DATETIME', 'DBSHUTDOWN_ABORT', 'DBSHUTDOWN_FINAL', 'DBSHUTDOWN_IMMEDIATE', 'DBSHUTDOWN_TRANSACTIONAL', 'DBSHUTDOWN_TRANSACTIONAL_LOCAL', 'DB_TYPE_BFILE', 'DB_TYPE_BINARY_DOUBLE', 'DB_TYPE_BINARY_FLOAT', 'DB_TYPE_BINARY_INTEGER', 'DB_TYPE_BLOB', 'DB_TYPE_BOOLEAN', 'DB_TYPE_CHAR', 'DB_TYPE_CLOB', 'DB_TYPE_CURSOR', 'DB_TYPE_DATE', 'DB_TYPE_INTERVAL_DS', 'DB_TYPE_INTERVAL_YM', 'DB_TYPE_JSON', 'DB_TYPE_LONG', 'DB_TYPE_LONG_RAW', 'DB_TYPE_NCHAR', 'DB_TYPE_NCLOB', 'DB_TYPE_NUMBER', 'DB_TYPE_NVARCHAR', 'DB_TYPE_OBJECT', 'DB_TYPE_RAW', 'DB_TYPE_ROWID', 'DB_TYPE_TIMESTAMP', 'DB_TYPE_TIMESTAMP_LTZ', 'DB_TYPE_TIMESTAMP_TZ', 'DB_TYPE_VARCHAR', 'DEFAULT_AUTH', 'DEQ_BROWSE', 'DEQ_FIRST_MSG', 'DEQ_IMMEDIATE', 'DEQ_LOCKED', 'DEQ_NEXT_MSG', 'DEQ_NEXT_TRANSACTION', 'DEQ_NO_WAIT', 'DEQ_ON_COMMIT', 'DEQ_REMOVE', 'DEQ_REMOVE_NODATA', 'DEQ_WAIT_F

In [19]:
# 1. 여러건 추가
dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
conn = cx_Oracle.connect('scott', 'tiger', dsn)
cursor = conn.cursor()

items = [
    (5, '레몬', 1500, '레몬에 포함된 구연산은 피로회복에... ', 'lemon.jpg'),
    (6, '오렌지', 2000, '오렌지에 포함된 구연산은 피로회복에. ', 'orange.jpg'),
    (7, '키위', 2500, '키위에 포함된 구연산은 피로회복에... ', 'kiwi.jpg'),
    (8, '포도', 3000, '포도에 포함된 구연산은 피로회복에... ', 'grape.jpg')    
]

sql = 'insert into product values(:1, :2, :3, :4, :5)'

for item in items:
    cursor.execute(sql, item)

conn.commit()
# cursor.close()
conn.close()

In [22]:
# 2. select(1)
dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
conn = cx_Oracle.connect('scott', 'tiger', dsn)
cursor = conn.cursor()

sql = 'select * from product order by product_id'
cursor.execute(sql)

for row in cursor:
    # clob필드는 어떻게 처리되는지?
    print(type(row), row)
print()

cursor.execute(sql)
for row in cursor:
    print(row[0],row[1],row[2],row[3],row[4], end='\n')
print()

cursor.execute(sql)
for row in cursor:
    # clob필드를 읽는 방법
    print(row[3].read())
print()

cursor.close()
conn.close()

<class 'tuple'> (1, '레몬', 1500, <cx_Oracle.LOB object at 0x00000139BCC3F090>, 'lemon.jpg')
<class 'tuple'> (2, '오렌지', 2000, <cx_Oracle.LOB object at 0x00000139BCB53540>, 'orange.jpg')
<class 'tuple'> (3, '키위', 2500, <cx_Oracle.LOB object at 0x00000139BCC3F090>, 'kiwi.jpg')
<class 'tuple'> (4, '포도', 3000, <cx_Oracle.LOB object at 0x00000139BCB53540>, 'grape.jpg')
<class 'tuple'> (5, '레몬', 1500, <cx_Oracle.LOB object at 0x00000139BCC3F090>, 'lemon.jpg')
<class 'tuple'> (6, '오렌지', 2000, <cx_Oracle.LOB object at 0x00000139BCB53540>, 'orange.jpg')
<class 'tuple'> (7, '키위', 2500, <cx_Oracle.LOB object at 0x00000139BCC3F090>, 'kiwi.jpg')
<class 'tuple'> (8, '포도', 3000, <cx_Oracle.LOB object at 0x00000139BCB53540>, 'grape.jpg')

1 레몬 1500 레몬에 포함된 구연산은 피로회복에...  lemon.jpg
2 오렌지 2000 오렌지에 포함된 구연산은 피로회복에.  orange.jpg
3 키위 2500 키위에 포함된 구연산은 피로회복에...  kiwi.jpg
4 포도 3000 포도에 포함된 구연산은 피로회복에...  grape.jpg
5 레몬 1500 레몬에 포함된 구연산은 피로회복에...  lemon.jpg
6 오렌지 2000 오렌지에 포함된 구연산은 피로회복에.  orange.jpg
7 키위 2500 

In [25]:
# 2. select(2) - 전체 조회후 list로 저장
dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
conn = cx_Oracle.connect('scott', 'tiger', dsn)
cursor = conn.cursor()

sql = 'select * from product order by product_id'
cursor.execute(sql)

rows = cursor.fetchall() # list로 리턴
print(type(rows), rows)
print()

for row in rows:
    desc = row[3].read()
    print(row)
    print(f'... {desc}')

cursor.close()
conn.close()

<class 'list'> [(1, '레몬', 1500, <cx_Oracle.LOB object at 0x00000139BCE216B0>, 'lemon.jpg'), (2, '오렌지', 2000, <cx_Oracle.LOB object at 0x00000139BCE219B0>, 'orange.jpg'), (3, '키위', 2500, <cx_Oracle.LOB object at 0x00000139BCC237B0>, 'kiwi.jpg'), (4, '포도', 3000, <cx_Oracle.LOB object at 0x00000139BCC234B0>, 'grape.jpg'), (5, '레몬', 1500, <cx_Oracle.LOB object at 0x00000139BCC23960>, 'lemon.jpg'), (6, '오렌지', 2000, <cx_Oracle.LOB object at 0x00000139BCC22CD0>, 'orange.jpg'), (7, '키위', 2500, <cx_Oracle.LOB object at 0x00000139BCC23900>, 'kiwi.jpg'), (8, '포도', 3000, <cx_Oracle.LOB object at 0x00000139BCC23480>, 'grape.jpg')]

(1, '레몬', 1500, <cx_Oracle.LOB object at 0x00000139BCE216B0>, 'lemon.jpg')
... 레몬에 포함된 구연산은 피로회복에... 
(2, '오렌지', 2000, <cx_Oracle.LOB object at 0x00000139BCE219B0>, 'orange.jpg')
... 오렌지에 포함된 구연산은 피로회복에. 
(3, '키위', 2500, <cx_Oracle.LOB object at 0x00000139BCC237B0>, 'kiwi.jpg')
... 키위에 포함된 구연산은 피로회복에... 
(4, '포도', 3000, <cx_Oracle.LOB object at 0x00000139BCC234B0>, 'grap

In [26]:
# select(3) - fetchone
dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
conn = cx_Oracle.connect('scott', 'tiger', dsn)
cursor = conn.cursor()

sql = 'select count(*) from product'
cursor.execute(sql)
count = cursor.fetchone()
print(type(count), count)

cursor.close()
conn.close()

<class 'tuple'> (8,)


In [29]:
# 4. update
dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
conn = cx_Oracle.connect('scott', 'tiger', dsn)
cursor = conn.cursor()

sql = 'update product set procuct_name = :1 where product_id = :2'
cursor.execute(sql, ('사과', 6))
conn.commit()

cursor.close()
conn.close()

In [31]:
# 5. delete
dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
conn = cx_Oracle.connect('scott', 'tiger', dsn)
cursor = conn.cursor()

cursor.execute('delete from product')
conn.commit()

cursor.close()
conn.close()

### C. SQLite
* 파일기반 소용량의 데이터베이스 시스템
* 공식사이트 : http://sqlite.com
* 개발자
  - 리차드 힙이 2008.8월에 발표(C로 개발)
  - 최신버전 : 3.45.2 (2024-03-12)
* SQLite의 개요
  - 파일기반 RDMBS, 저메모리, 빠른 속도
  - Open Source
  - 별도의 DB서버가 없어도 쉽고 빠르게 사용할 수 있는 `embeded SQL DB엔진`
  - 언드로이드, 아이폰등 스마트기기의 내장DB로 많이 사용된다.
  - `표준SQL을 지원`
 
* SQLite에서 지원되지 않는 기능
  - https://www.sqlite.com/omitted.html
* SQLite 클라이언트 tool
  - http://www.sqliteexpert.com
  - personal edition 64bit download and install
* python에서 `SQLite3은 파이썬의 표준라이브러리이기 때문에 별도 설치 없이도 사용`할 수 있다.

In [1]:
import sqlite3
print(f'sqlite3 모듈 버전 : {sqlite3.version}')
print(f'sqlite3 DB 버전 : {sqlite3.sqlite_version}')
sqlite3.sqlite_version_info

sqlite3 모듈 버전 : 2.6.0
sqlite3 DB 버전 : 3.41.2


(3, 41, 2)

#### 1. table 생성

In [2]:
def create_table(table):
    # db connection
    # 해당폴더에 db가 없으면 connection할 때 자동으로 생성
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()

    # 테이블생성
    # 제목, 출판일, 출판사, 페이지수, 추천여부
    cursor.execute(f'''
        create table if not exists {table} (
              title            text
            , published_date   text
            , publisher        text
            , pages            integer
            , recommand        integer
        )
    ''')

    conn.commit()
    cursor.close()
    conn.close()

create_table('books')

#### 2. insert

In [3]:
# 1. 1건 입력
def insert_table():
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        insert into books values('Python', '2024-03-22', '이젠', 580, 10)
    ''')

    conn.commit()
    cursor.close()
    conn.close()

insert_table()

In [4]:
# 2. tuple을 이용한 1건 입력
def insert_book_1():
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()
    
    sql = 'insert into books values(?,?,?,?,?)'
    cursor.execute(sql, ('Java', '2024-03-22', '한빛미디어', 400, 5))

    conn.commit()
    cursor.close()
    conn.close()

insert_book_1()

In [5]:
# 3. tuple을 이용한 여러 건 입력
def insert_book_2():
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()
    
    sql = 'insert into books values(?,?,?,?,?)'

    # 책의 정보를 저장한 list(요소는 tuple)
    books = [
        ('Big Data', '2024-03-22', '길벗', 400, 5),
        ('ChatBot', '2024-03-22', '제이펍', 400, 5),
        ('React.js', '2024-03-22', '교보문고', 400, 5),
        ('Node.js', '2024-03-22', '위키북스', 400, 5)
    ]

    # 여러건 executemany()
    cursor.executemany(sql, books)
         
    conn.commit()
    cursor.close()
    conn.close()

insert_book_2()

##### 3. select

In [6]:
# 1. 전체 도서목록 조회 - fetchall()
def all_books():
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()

    sql = 'select * from books'
    cursor.execute(sql)

    books = cursor.fetchall()

    for book in books:
        print(type(book), book)
    
    cursor.close()
    conn.close()

all_books()

<class 'tuple'> ('Python', '2024-03-22', '이젠', 580, 10)
<class 'tuple'> ('Java', '2024-03-22', '한빛미디어', 400, 5)
<class 'tuple'> ('Big Data', '2024-03-22', '길벗', 400, 5)
<class 'tuple'> ('ChatBot', '2024-03-22', '제이펍', 400, 5)
<class 'tuple'> ('React.js', '2024-03-22', '교보문고', 400, 5)
<class 'tuple'> ('Node.js', '2024-03-22', '위키북스', 400, 5)
<class 'tuple'> ('Python', '2024-03-22', '이젠', 580, 10)
<class 'tuple'> ('Java', '2024-03-22', '한빛미디어', 400, 5)
<class 'tuple'> ('Big Data', '2024-03-22', '길벗', 400, 5)
<class 'tuple'> ('ChatBot', '2024-03-22', '제이펍', 400, 5)
<class 'tuple'> ('React.js', '2024-03-22', '교보문고', 400, 5)
<class 'tuple'> ('Node.js', '2024-03-22', '위키북스', 400, 5)


In [7]:
# 2. 일부자료조회 - fetchmany(n)
def some_books(count):
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()

    sql = 'select * from books'
    cursor.execute(sql)

    # 일부자료 fetchmany(number) : number갯수만큼 리턴
    print('\n일부자료조회하기')
    print('-'*60)

    books = cursor.fetchmany(count)

    for book in books:
        print(type(book), book)
    
    cursor.close()
    conn.close()

some_books(2)
some_books(5)


일부자료조회하기
------------------------------------------------------------
<class 'tuple'> ('Python', '2024-03-22', '이젠', 580, 10)
<class 'tuple'> ('Java', '2024-03-22', '한빛미디어', 400, 5)

일부자료조회하기
------------------------------------------------------------
<class 'tuple'> ('Python', '2024-03-22', '이젠', 580, 10)
<class 'tuple'> ('Java', '2024-03-22', '한빛미디어', 400, 5)
<class 'tuple'> ('Big Data', '2024-03-22', '길벗', 400, 5)
<class 'tuple'> ('ChatBot', '2024-03-22', '제이펍', 400, 5)
<class 'tuple'> ('React.js', '2024-03-22', '교보문고', 400, 5)


In [8]:
# 3. 한 개의 자료만 조회 - fetchone()
def one_book():
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()

    sql = 'select * from books'
    cursor.execute(sql)
    
    # 한 개의 자료 fetchone()
    print('\n일부자료조회하기')
    print('-'*60)

    book = cursor.fetchone()
    # print(type(book), book, '\n')
    
    for col in book:
        print(type(col), col)
    
    cursor.close()
    conn.close()
    
one_book()


일부자료조회하기
------------------------------------------------------------
<class 'str'> Python
<class 'str'> 2024-03-22
<class 'str'> 이젠
<class 'int'> 580
<class 'int'> 10


In [9]:
# 4. 조건별 조회 : page > 500
def big_books(pages):
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()

    sql = f'select * from books where pages > {pages}'  # + str(pages)
    cursor.execute(sql)

    print(f'페이지수가 {pages}보다 큰 책 조회하기')
    print('-'*60)

    books = cursor.fetchall()

    for book in books:
        print(book)
        
    cursor.close()
    conn.close()
    
big_books(300)

페이지수가 300보다 큰 책 조회하기
------------------------------------------------------------
('Python', '2024-03-22', '이젠', 580, 10)
('Java', '2024-03-22', '한빛미디어', 400, 5)
('Big Data', '2024-03-22', '길벗', 400, 5)
('ChatBot', '2024-03-22', '제이펍', 400, 5)
('React.js', '2024-03-22', '교보문고', 400, 5)
('Node.js', '2024-03-22', '위키북스', 400, 5)
('Python', '2024-03-22', '이젠', 580, 10)
('Java', '2024-03-22', '한빛미디어', 400, 5)
('Big Data', '2024-03-22', '길벗', 400, 5)
('ChatBot', '2024-03-22', '제이펍', 400, 5)
('React.js', '2024-03-22', '교보문고', 400, 5)
('Node.js', '2024-03-22', '위키북스', 400, 5)


#### 4. update

In [10]:
# 1. Python책의 페이지수를 680, 추천수를 5으로 수정
def update_books():
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()

    sql = 'update books set pages=?, recommand=? where title=?'
    cursor.execute(sql, (680, 5, 'Python'))

    conn.commit()    
    cursor.close()
    conn.close()
    
update_books()
all_books()

<class 'tuple'> ('Python', '2024-03-22', '이젠', 680, 5)
<class 'tuple'> ('Java', '2024-03-22', '한빛미디어', 400, 5)
<class 'tuple'> ('Big Data', '2024-03-22', '길벗', 400, 5)
<class 'tuple'> ('ChatBot', '2024-03-22', '제이펍', 400, 5)
<class 'tuple'> ('React.js', '2024-03-22', '교보문고', 400, 5)
<class 'tuple'> ('Node.js', '2024-03-22', '위키북스', 400, 5)
<class 'tuple'> ('Python', '2024-03-22', '이젠', 680, 5)
<class 'tuple'> ('Java', '2024-03-22', '한빛미디어', 400, 5)
<class 'tuple'> ('Big Data', '2024-03-22', '길벗', 400, 5)
<class 'tuple'> ('ChatBot', '2024-03-22', '제이펍', 400, 5)
<class 'tuple'> ('React.js', '2024-03-22', '교보문고', 400, 5)
<class 'tuple'> ('Node.js', '2024-03-22', '위키북스', 400, 5)


#### 5. delete

In [11]:
# 2. Python책을 삭제
def delete_book():
    conn = sqlite3.connect('./data/mybook.db')
    cursor = conn.cursor()
    
    sql = 'delete from books where title=?'
    cursor.execute(sql, ('Python',))
    
    conn.commit()
    cursor.close()
    conn.close()
    
delete_book()
all_books()

<class 'tuple'> ('Java', '2024-03-22', '한빛미디어', 400, 5)
<class 'tuple'> ('Big Data', '2024-03-22', '길벗', 400, 5)
<class 'tuple'> ('ChatBot', '2024-03-22', '제이펍', 400, 5)
<class 'tuple'> ('React.js', '2024-03-22', '교보문고', 400, 5)
<class 'tuple'> ('Node.js', '2024-03-22', '위키북스', 400, 5)
<class 'tuple'> ('Java', '2024-03-22', '한빛미디어', 400, 5)
<class 'tuple'> ('Big Data', '2024-03-22', '길벗', 400, 5)
<class 'tuple'> ('ChatBot', '2024-03-22', '제이펍', 400, 5)
<class 'tuple'> ('React.js', '2024-03-22', '교보문고', 400, 5)
<class 'tuple'> ('Node.js', '2024-03-22', '위키북스', 400, 5)


##### SQLite를 RAM에서 직접사용하기

* sqlite을 RAM에서 사용하려면 `:memory:`로 연결하면 된다.
* 장점은 RAM에서 실행되기 때문에 처리속도가 빠르다. 하지만
* 단점은 휘발성이기 때문에 저장되지 않는다.

In [15]:
import sqlite3

conn = sqlite3.connect(':memory:')
sql = '''
    create table if not exists sales(
          customer    varchar(20)
        , product     varchar(20)
        , price       float
        , order_date  date
    )
'''
conn.execute(sql)

data = [
    ('소향', '배', 9000, '2024-03-22'),
    ('나얼', '사과', 10000, '2024-03-22'),
    ('거미', '체리 1kg', 8000, '2024-03-22'),
    ('효신', '딸기(1kg)', 7000, '2024-03-22'),
]

sql = 'insert into sales values(?,?,?,?)'
conn.executemany(sql, data)
conn.commit()

cursor = conn.execute('select * from sales')
rows = cursor.fetchall()

count = 0
for row in rows:
    print(row)
    count += 1

print(f'총처리건수 = {count}')
cursor.close()
conn.close()

('소향', '배', 9000.0, '2024-03-22')
('나얼', '사과', 10000.0, '2024-03-22')
('거미', '체리 1kg', 8000.0, '2024-03-22')
('효신', '딸기(1kg)', 7000.0, '2024-03-22')
총처리건수 = 4


##### 연습문제 - 판매관리DB(sqlite, oracle, mariadb)

1. ./data/db/mysales.db 라는 db를 생성하세요
2. sales 테이블 생성하기
   customer varchar(20)
   product varchar(20)
   price float
   date date
   함수이름 : create_table_sales()
   
3. 테이블조회 : show_tables()
   1) sqlite_master 시스템테이블
   2) type = 'table'
   3) for table in tables:

4. 데이터추가 : insert_sales()
   1) 자료추가 : 리스트 = 테이블, 튜플 = 레코드
   2) 4건정도 추가

5. 데이터조회 : select_sales()
   1) 추가한 데이터 조회